In [1]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
w = widgets.IntSlider(description='Img idx', continuous_update=False)
w1 = widgets.IntSlider(description='bbox threshold', continuous_update=False)
w.value=50 # 처음 값 조정
widgets.link((w, 'value'), (w1, 'value'))
widgets.VBox([w, w1])

In [3]:

@widgets.interact_manual(color=['blue', 'red', 'green'], lw=(1.0,10.0))
def plot(freq=1, color='blue', lw=2, grid=True):
    
    t = np.linspace(-1, 1, 1000)
    fig, ax = plt.subplots(1, 1)
    ax.plot(t, np.sin(2*np.pi*freq*t), lw=lw, color=color)
    ax.grid(grid)

interactive(children=(IntSlider(value=1, description='freq', max=3, min=-1), Dropdown(description='color', opt…

In [4]:
button = widgets.Button(description='save image')
output = widgets.Output()
def clicked(b):
    with output:
        print('hi')
@interact(idx=(0, 10), threshold=(0, 10))
def showImg(idx=0, threshold=0):
    display(button, output)
    button.on_click(clicked)
    print(idx, threshold)

interactive(children=(IntSlider(value=0, description='idx', max=10), IntSlider(value=0, description='threshold…

In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [2]:
def makeGroundTruthDF(annotation_path):
    coco = COCO(annotation_path)

    img_id = coco.getImgIds()
    img_info = coco.loadImgs(img_id)
    fnames = [info['file_name'] for info in img_info]
    ids = [info['id'] for info in img_info]
    
    gt_string = []
    for id in ids:
        ann_id = coco.getAnnIds(imgIds=id)
        annotation_info_list = coco.loadAnns(ann_id)

        string_temp = ""
        fname_idx = annotation_info_list[0]['image_id'] # img_id index
        for ann_info in annotation_info_list:
            category_id = ann_info['category_id']
            x,y,w,h = ann_info['bbox'] # bbox coordinate
            x,y,x1,y1 = str(x), str(y), str(x+w), str(y+h)
            string_temp += (str(category_id) + ' ' + x + ' ' + y + ' ' + x1 + ' ' + y1 + ' ')
            
        gt_string.append(string_temp)
    return pd.DataFrame(data=list(zip(gt_string, fnames)), columns=['GroundTruthString','image_id'])

gt_df = makeGroundTruthDF('/opt/ml/detection/dataset/train.json')
gt_df.head()

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


,GroundTruthString,image_id
0,0 197.6 193.7 745.4 663.4,train/0000.jpg
1,3 0.0 407.4 57.6 588.0 7 0.0 455.6 144.6 637.2...,train/0001.jpg
2,3 267.9 165.2 899.5 678.2,train/0002.jpg
3,2 462.2 369.4 696.1 624.0 6 773.3 3.0 961.6999...,train/0003.jpg
4,1 567.5 462.2 732.7 551.6 1 859.4 411.7 1023.5...,train/0004.jpg


In [3]:
# 같은 class에 대해서 confidence 기준 정렬
# 그 중 IoU가 가장 큰 Bbox를 정답으로 여김
submission = pd.read_csv('/opt/ml/detection/baseline/faster_rcnn/faster_rcnn_torchvision_submission_2.csv')

def IOU(gt, pred):
    gt = [float(gt[0]), float(gt[1]), float(gt[2]), float(gt[3])]
    pred = [float(pred[0]), float(pred[1]), float(pred[2]), float(pred[3])]

    gt_area = (gt[2]-gt[0])*(gt[3]-gt[1])
    pred_area = (pred[2]-pred[0])*(pred[3]-pred[1])
    
    x = max(gt[0], pred[0])
    y = max(gt[1], pred[1])
    x1 = min(gt[2], pred[2])
    y1 = min(gt[3], pred[3])

    w = max(0, x1-x)
    h = max(0, y1-y)

    inter = w*h
    iou = inter / (gt_area + pred_area - inter)
    return iou


def matchBbox(gt_df, predict_df, iou_threshold=0):
    wrong_per_cls = [0 for _ in range(10)]

    for i in range(len(predict_df)):
        try:
            predict_string = np.asarray(predict_df.iloc[i]['PredictionString'].split(' ')[:-1]).reshape(-1, 6)
            fname = predict_df.iloc[i]['image_id']
            gt_string = np.asarray(gt_df.iloc[i]['GroundTruthString'].split(' ')[:-1]).reshape(-1, 5)
            check = [False for _ in range(len(gt_string))]
            for pred_row in predict_string:
                pred_cls = pred_row[0]
                pred_conf = pred_row[1]
                max_iou = 0
                correct_idx = -1
                for idx, gt_row in enumerate(gt_string):
                    if(check[idx]):
                        continue
                    gt_cls = gt_row[0]
                    iou = IOU(gt_row[1:], pred_row[2:])
                    if(gt_cls==pred_cls and iou >= iou_threshold and max_iou < iou):
                        max_iou = iou
                        correct_idx = idx
                check[correct_idx]=True

            for idx, gt_row in enumerate(gt_string):
                if(not check[idx]):
                    wrong_per_cls[int(gt_row[0])]+=1

        except:
            gt_string = np.asarray(gt_df.iloc[i]['GroundTruthString'].split(' ')[:-1]).reshape(-1, 5)
            for gt_row in gt_string:
                wrong_per_cls[int(gt_row[0])]+=1

            img_id = predict_df.iloc[i]['image_id']
            print(f'no boxes found in {img_id}!')

            pass
    return wrong_per_cls
matchBbox(gt_df, submission)

no boxes found in test/4194.jpg!


[1608, 3944, 504, 471, 761, 1769, 1082, 2719, 121, 298]

In [ ]:
def makeGroundTruthDF(annotation_path):
    coco = COCO(annotation_path)

    img_id = coco.getImgIds()
    img_info = coco.loadImgs(img_id)
    fnames = [info['file_name'] for info in img_info]
    ids = [info['id'] for info in img_info]
    
    gt_string = []
    for id in ids:
        ann_id = coco.getAnnIds(imgIds=id)
        annotation_info_list = coco.loadAnns(ann_id)
        
        string_temp = ""
        fname_idx = annotation_info_list[0]['image_id'] # img_id index
        for ann_info in annotation_info_list:
            category_id = ann_info['category_id']
            x,y,w,h = ann_info['bbox'] # bbox coordinate
            x,y,x1,y1 = str(x), str(y), str(x+w), str(y+h)
            string_temp += (str(category_id) + ' ' + x + ' ' + y + ' ' + x1 + ' ' + y1 + ' ')
            
        gt_string.append(string_temp)
    return pd.DataFrame(data=list(zip(gt_string, fnames)), columns=['GroundTruthString','image_id'])

gt_df = makeGroundTruthDF('/opt/ml/detection/dataset/train.json')
gt_df.head()

In [ ]:
# 같은 class에 대해서 confidence 기준 정렬
# 그 중 IoU가 가장 큰 Bbox를 정답으로 여김
submission = pd.read_csv('/opt/ml/detection/baseline/mmdetection/work_dirs/yolov3_d53_320_273e_coco_custom_trash/submission_epoch_15.csv')

def IOU(gt, pred):
    gt = [float(gt[0]), float(gt[1]), float(gt[2]), float(gt[3])]
    pred = [float(pred[0]), float(pred[1]), float(pred[2]), float(pred[3])]

    gt_area = (gt[2]-gt[0])*(gt[3]-gt[1])
    pred_area = (pred[2]-pred[0])*(pred[3]-pred[1])
    
    x = max(gt[0], pred[0])
    y = max(gt[1], pred[1])
    x1 = min(gt[2], pred[2])
    y1 = min(gt[3], pred[3])

    w = max(0, x1-x)
    h = max(0, y1-y)

    inter = w*h
    iou = inter / (gt_area + pred_area - inter)
    return iou


def matchBbox(gt_df, predict_df, iou_threshold=0):
    wrong_per_cls = [0 for _ in range(10)]
    # class별로 못 맞춘 개수 저장하는 변수
    
    total_img = len(predict_df)

    img_pred_gt_match = [[] for _ in range(total_img)]
    # 이미지마다 predict와 gt를 matching하고 맞춘 predict, gt를 저장하기 위한 변수
    
    for i in range(total_img):
        # 모든 이미지에 대해 검사
        try:
            fname = predict_df.iloc[i]['image_id']
            predict_string = np.asarray(predict_df.iloc[i]['PredictionString'].split(' ')[:-1]).reshape(-1, 6)
            gt_string = np.asarray(gt_df.iloc[i]['GroundTruthString'].split(' ')[:-1]).reshape(-1, 5)
            num_gt = len(gt_string)
            num_pred = len(predict_string)

            check = [False for _ in range(num_gt)]
            # predict한 bbox와 matching된 ground_truth 확인하기
            # matching된 게 있으면 True, 없으면 False저장

            gt_pred_match = []
            # 각각의 predict한 bbox와 가장 잘 맞는 gt bbox 저장하기 위한 리스트

            for pred_idx, pred_row in enumerate(predict_string):
                pred_cls = pred_row[0]
                pred_conf = pred_row[1]

                max_iou = 0
                matched_gt_idx = -1
                # predict한 bbox와 전체 gt와 비교하여 가장 잘 맞은 gt idx를 저장하기 위한 변수

                for gt_idx, gt_row in enumerate(gt_string):
                    
                    iou = IOU(gt_row[1:], pred_row[2:])
                    # predict bbox와 gt bbox IOU 계산
                    gt_cls = gt_row[0]
                    if(gt_cls==pred_cls and iou >= iou_threshold and max_iou < iou):
                        # 만약 class가 동일하고, IOU가 threshold보다 크며, 이전까지 가장 잘 맞는 IOU보다 현재가 더 큰 경우 갱신
                        max_iou = iou
                        matched_gt_idx = gt_idx

                if(matched_gt_idx!=-1):
                    # threshold이상으로 잘 맞는 gt bbox가 있다면 저장
                    
                    gt_pred_match.append([pred_idx, matched_gt_idx, max_iou])
                    # 이미지에서 predict한 bbox index, 가장 잘 맞는 gt index, 해당 IOU

                    check[matched_gt_idx]=True
                    # predict한 bbox가 gt를 잘 맞췄으니, 해당 gt를 True로 변경
                    # (못 맞춘 gt 개수 파악을 위한 용도)

            for idx, gt_row in enumerate(gt_string):
                if(not check[idx]):
                    # 만약 gt와 맞는 prediction이 없다면
                    wrong_per_cls[int(gt_row[0])]+=1
                    # 해당 gt의 class는 못 맞춘 것이므로 1증가
            
            img_pred_gt_match[i].append(gt_pred_match)
            # 해당 이미지의 gt_pred_match를 저장
            # 시각화할 때 편하려고 만듬

        except:
            # 예측한 bbox가 없는 경우
            gt_string = np.asarray(gt_df.iloc[i]['GroundTruthString'].split(' ')[:-1]).reshape(-1, 5)
            for gt_row in gt_string:
                wrong_per_cls[int(gt_row[0])]+=1
            # 모든 gt를 못 맞췄으므로 해당 gt class마다 1증가

            img_id = predict_df.iloc[i]['image_id']
            print(f'no boxes found in {img_id}!')

            pass
    return img_pred_gt_match, wrong_per_cls
img_pred_gt_match, wrong_per_cls = matchBbox(gt_df, submission)
print(wrong_per_cls)
print(img_pred_gt_match[0])

In [ ]:
from PIL import Image
from ipywidgets import interact
import skimage.io as io
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
import matplotlib.patches as patches
import seaborn as sns
import ipywidgets as widgets


num_img = len(submission)

def showBoxes(img, boxes, ax, gt=True, clicked=-1):
    # gt와 confidence score가 없으므로 argument로 넣어줌

    palette = sns.color_palette('bright')
    ax.imshow(img)
    if gt:
        for i in range(0, len(boxes)):
            class_idx = int(boxes[i][0])
            color = palette[class_idx]
            x,y,x1,y1 = boxes[i][1:5]
            x,y,x1,y1 = float(x), float(y), float(x1), float(y1)
            ax.add_patch(
                patches.Rectangle(
                    (x,y), x1-x, y1-y,
                    edgecolor=color,
                    linewidth=1,
                    fill=False,
                    ),
                )
            text_y = y-20 if y>20 else y+20 
            ax.text(x+18,text_y, f'{classes[class_idx]}', color='white', fontsize='5', weight='semibold', backgroundcolor=color)
            ax.set_xticks([])
            ax.set_yticks([])
    else:
        for i in range(0, len(boxes)):
            class_idx = int(boxes[i][0])
            color = palette[class_idx]
            conf = float(boxes[i][1])
            x,y,x1,y1 = boxes[i][2:6]
            x,y,x1,y1 = float(x), float(y), float(x1), float(y1)
            ax.add_patch(
                patches.Rectangle(
                    (x,y), x1-x, y1-y,
                    edgecolor=color,
                    linewidth=1,
                    fill=False,
                    ),
                )
            text_y = y-20 if y>20 else y+20 
            ax.text(x+18,text_y, f'{classes[class_idx]}({conf:.2f})', color='white', fontsize='5', weight='semibold', backgroundcolor=color)
            ax.set_xticks([])
            ax.set_yticks([])
    
@interact(idx=(0, num_img-1))
def showImg(idx):
    fig, ax = plt.subplots(1, 3, dpi=150)
    img = io.imread(os.path.join('/opt/ml/detection/dataset', submission.iloc[idx]['image_id']))
    boxes_pred = np.asarray(submission.iloc[idx]['PredictionString'].split(' ')[:-1]).reshape(-1, 6)
    boxes_gt = np.asarray(gt_df.iloc[idx]['GroundTruthString'].split(' ')[:-1]).reshape(-1, 5)

    total_idx = np.ones(len(boxes_pred))
    
    try:
        positive_box_idx = np.array(img_pred_gt_match[idx])[...,0].squeeze().astype(int)
        boxes_pos = boxes_pred[positive_box_idx]
        total_idx[positive_box_idx]=0
        showBoxes(boxes_pos, ax[1])
    except:
        ax[1].imshow(img)  
        ax[1].set_xticks([])
        ax[1].set_yticks([])

    negative_box_idx = np.where(total_idx==1)[0]
    # 좀 조잡한데, positive랑 negative index 추출
    boxes_neg = boxes_pred[negative_box_idx]
        # positive, negative box 추출
    ax[0].set_title('GT')
    ax[1].set_title('Pos')
    ax[2].set_title('Neg')
    showBoxes(img, boxes_gt, ax[0], gt=True, clicked=-1)
    showBoxes(img, boxes_neg, ax[2])